# AlexNet Architecture

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%%capture
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPooling2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from time import time
import numpy as np

In [3]:
train_data=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
training_set=train_data.flow_from_directory('Dataset/Train',target_size=(224,224),batch_size=32,class_mode='categorical')

Found 536 images belonging to 6 classes.


In [4]:
test_data=ImageDataGenerator(rescale=1./255)
test_set=test_data.flow_from_directory('Dataset/Test',target_size=(224,224),batch_size=32,class_mode='categorical')

Found 493 images belonging to 6 classes.


In [5]:
model = Sequential()
# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# 2nd Fully Connected Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# 3rd Fully Connected Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Output Layer
model.add(Dense(6))
model.add(Activation('softmax'))
model.summary()

# Compile the model
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 activation (Activation)     (None, 54, 54, 96)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 17, 17, 256)       2973952   
                                                                 
 activation_1 (Activation)   (None, 17, 17, 256)       0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 256)        0         
 2D)                                                    

In [6]:
model_path = "AlexNet.h5"

callbacks = [
    ModelCheckpoint(model_path, monitor='accuracy', verbose=1, save_best_only=True)
]

In [7]:
epochs = 100
batch_size = 32

In [ ]:
#### Fitting the model
history = model.fit(
           training_set, steps_per_epoch=training_set.samples // batch_size, 
           epochs=epochs, 
           validation_data=test_set,validation_steps=test_set.samples // batch_size,
           callbacks=callbacks)

Epoch 1/100
16/16 [==============================] - ETA: 0s - loss: 1.8026 - accuracy: 0.3214
Epoch 1: accuracy improved from -inf to 0.32143, saving model to AlexNet.h5
16/16 [==============================] - 64s 4s/step - loss: 1.8026 - accuracy: 0.3214 - val_loss: 1.5942 - val_accuracy: 0.3979
Epoch 2/100
16/16 [==============================] - ETA: 0s - loss: 1.6147 - accuracy: 0.3810
Epoch 2: accuracy improved from 0.32143 to 0.38095, saving model to AlexNet.h5
16/16 [==============================] - 59s 4s/step - loss: 1.6147 - accuracy: 0.3810 - val_loss: 1.5814 - val_accuracy: 0.3958
Epoch 3/100
16/16 [==============================] - ETA: 0s - loss: 1.5927 - accuracy: 0.3869
Epoch 3: accuracy improved from 0.38095 to 0.38690, saving model to AlexNet.h5
16/16 [==============================] - 63s 4s/step - loss: 1.5927 - accuracy: 0.3869 - val_loss: 1.5770 - val_accuracy: 0.3917
Epoch 4/100
16/16 [==============================] - ETA: 0s - loss: 1.5764 - accuracy: 0.3906

16/16 [==============================] - 56s 4s/step - loss: 1.5901 - accuracy: 0.3790 - val_loss: 1.5752 - val_accuracy: 0.3917
Epoch 31/100
15/16 [===========================>..] - ETA: 1s - loss: 1.5828 - accuracy: 0.3938

In [ ]:
import matplotlib.pyplot as plt
def graph():
    #Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
graph()
   

In [ ]:
def graphs():
# Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
graphs()